In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from numpy.random import seed
import tensorflow as tf
import sklearn
from sklearn import preprocessing
from sklearn import model_selection
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.python.keras.layers import GRU,Masking,TimeDistributed
from tensorflow.python.keras.layers.core import Dense, Dropout, Activation,Flatten
from tensorflow.python.keras.layers.merge import concatenate, Average  
from tensorflow.python.keras.layers.embeddings import Embedding
from tensorflow.python.keras import regularizers,optimizers,backend,Input, Model
import tensorflow.python.keras.backend as K
import math
import sys
from tcn import TCN, tcn_full_summary

In [5]:
Columns = ["P_ID","5-1_value","5-2_value","5-3_value","2-1_M","2-3_M","3-1_M","5-1_M","5-3_M","TYPE"]
Dp = ['DATE','MASK_ID','SUBSTRATE_ID','SHUTTER','SRC3_SHUTTER_STATUS','EVAPORATION_HALT','TIME']
Label_Index = ['PLC_THICKNESS_SP','CRYSTAL_REMAINING_2','PLC_THICKNESS_1','CELL_CHANGE_CTRL_DLY_TIME_2','PLC_THICKNESS_2','THICKNESS_3','XXXX_POWER_3','PLC_THICKNESS_3']
TargetIndex = ["CIEX","CIEY","CIEX_DIFF","CIEY_DIFF"]

In [6]:
train_df = pd.read_csv("data1/train.csv")
train_df.drop_duplicates(subset = ['P_ID'],inplace=True)
train_log_df = pd.read_csv("data1/train_log.csv")
test_df = pd.read_csv("data1/test.csv")
test_log_df = pd.read_csv("data1/test_log.csv")
all_df = pd.concat([train_df,test_df])
all_log_df = pd.concat([train_log_df,test_log_df])
all_log_df = all_log_df.interpolate()
Data_df = all_df[Columns]

C:\Users\1911141\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\1911141\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [7]:
L = len(train_df)
OneHotData_df = pd.get_dummies(Data_df)
Std = ["5-1_value","5-2_value","5-3_value"]
OneHotData_df[Std] = pd.DataFrame(preprocessing.scale(OneHotData_df[Std]))
train = OneHotData_df[:L]
train = train.dropna()
sub_label = OneHotData_df[L:]
train_label, test_label, train_feature, test_feature =  model_selection.train_test_split( train,all_df[TargetIndex][:L], test_size=0.33, random_state=42)
train_label = train_label[:L]
sc = preprocessing.MinMaxScaler(feature_range=(0, 1)).fit(train_feature)
train_feature = pd.DataFrame(sc.transform(train_feature))
train_feature = train_feature[:L]

In [8]:
LOG = []
n=300
for i in test_df['P_ID']:
    C = all_log_df[all_log_df['P_ID']==i].sort_values('TIME')[Label_Index]
    X = []
    n = int(len(C)/40)
    for j in range(40):
        X.append(np.mean(C[j*n:(j+1)*n]))
    LOG.append(np.array(preprocessing.scale(X)))
    sys.stdout.write("\r Now Loading : %d " %(i))
    sys.stdout.flush()
sub_log = np.array(LOG)
LOG = []
n=300
for i in train_label['P_ID']:
    C = all_log_df[all_log_df['P_ID']==i].sort_values('TIME')[Label_Index]
    X = []
    n = int(len(C)/40)
    for j in range(40):
        X.append(np.mean(C[j*n:(j+1)*n]))
    LOG.append(np.array(preprocessing.scale(X)))
    #     LOG.append(np.array(C.drop(Dp,axis=1)[:41]))
    sys.stdout.write("\r Now Loading : %d " %(i))
    sys.stdout.flush()
train_log = np.array(LOG)
LOG = []
n=300
for i in test_label['P_ID']:
    C = all_log_df[all_log_df['P_ID']==i].sort_values('TIME')[Label_Index]
    X = []
    n = int(len(C)/40)
    for j in range(40):
        X.append(np.mean(C[j*n:(j+1)*n]))
    LOG.append(np.array(preprocessing.scale(X)))
    sys.stdout.write("\r Now Loading : %d " %(i))
    sys.stdout.flush()
test_log = np.array(LOG)

 Now Loading : 6339825 

In [ ]:
Model_Hybrid.load_weights("Model/GRU_MASKTEST.hdf5")
test_1_Pre =  Model_Hybrid.predict([test_label.drop(['P_ID'],axis=1),test_log])
Pre_df = pd.DataFrame(test_1_Pre)
Pre_df = pd.DataFrame(sc.inverse_transform(Pre_df))
W = [0.249,0.330,0.172,0.204]
Wmae = 0
for n in range(4):
    MAE = 0
    for i in range(len(Pre_df[0])):
        MAE = MAE + abs(Pre_df[n][i]-test_feature[TargetIndex[n]].iloc[i])
#     print(MAE/len(Pre_df[n]))
    Wmae = Wmae + (MAE/len(Pre_df[n]))*W[n]
print(Wmae)

In [ ]:
def my_mean_absolute_error(y_true, y_pred):
    W = [0.249,0.330,0.172,0.204]
    return K.sum((K.abs(y_pred - y_true)*W), axis=-1)

In [ ]:
backend.clear_session()
checkpointer = ModelCheckpoint(filepath='Model/GRU_ZIPTEST.hdf5',monitor='val_loss', verbose=1, save_best_only=True)
my_adam = optimizers.adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.01, amsgrad=False)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)

X_MLP = Input(shape=(35))
X_TS = Input(shape=(40,8))

MLP_Layer = Dense(128)(X_MLP)

# GRU_Layer = GRU(128,return_sequences=True)(X_TS)
# GRU_Layer = GRU(32,return_sequences=False)(GRU_Layer)

TCN_Layer = TCN(nb_filters=128, kernel_size=2, nb_stacks=1, dilations=[1, 2, 4, 8, 16, 32], padding='causal', use_skip_connections=False, dropout_rate=0.0, return_sequences=True, activation='relu', kernel_initializer='he_normal', use_batch_norm=False)(X_TS)
TCN_Layer = TCN(nb_filters=32,return_sequences=False)(TCN_Layer)

Merge = concatenate([MLP_Layer, GRU_Layer])
Merge = Dense(128, activation='relu')(Merge)
Y = Dense(4, activation='sigmoid')(Merge)


Model_Hybrid = Model(inputs=[X_MLP,X_TS] ,outputs = Y)
# TCN_Layer = Dense(128)(TCN(return_sequences=False)(X_TS))

Model_Hybrid.compile(loss=my_mean_absolute_error, optimizer=my_adam)

hist = Model_Hybrid.fit([train_label.drop(['P_ID'],axis=1),train_log], train_feature, batch_size=96,epochs=10000 , verbose=1,validation_split=0.1,callbacks=[checkpointer,callback])


In [36]:
checkpointer = ModelCheckpoint(filepath='SelectFeature.hdf5',monitor='val_loss', verbose=1, save_best_only=True)
my_adam = optimizers.adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.01, amsgrad=False)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)

Model_Hybrid.compile(loss=my_mean_absolute_error, optimizer=my_adam)
backend.clear_session()
Model_MLP = Sequential()
Model_MLP.add(Dense(128, kernel_initializer="uniform", input_dim=35,activation="relu"))
# Model_MLP.add(Dropout(0.2))
Model_GRU = Sequential()
# Model_GRU.add(Masking(mask_value=0., input_shape=(40, 8)))
Model_GRU.add(GRU(128, return_sequences=True,input_shape=(40, 8)))
Model_GRU.add(GRU(32))
# Model_GRU.add(Dropout(0.2))
Model_Hybrid = Sequential()
Model_Hybrid.add(Merge([Model_MLP,Model_GRU],mode = 'concat'))
Model_Hybrid.add(Dense(units=128,activation='relu'))
# Model_MLP.add(Dropout(0.2))
Model_Hybrid.add(Dense(units=4,activation='sigmoid'))
Model_Hybrid.summary()

# Model_Hybrid.load_weights("Model/weights.hdf5")
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)
hist = Model_Hybrid.fit([train_label.drop(['P_ID'],axis=1),train_log], train_feature, batch_size=96,epochs=10000 , verbose=1,validation_split=0.1,callbacks=[callback,checkpointer])
# Model_Hybrid.load_weights("Model/weights.hdf5")

C:\Users\1911141\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  # This is added back by InteractiveShellApp.init_path()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_5 (Merge)              (None, 160)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 128)               20608     
_________________________________________________________________
dense_17 (Dense)             (None, 4)                 516       
Total params: 93,796
Trainable params: 93,796
Non-trainable params: 0
_________________________________________________________________
Train on 2986 samples, validate on 332 samples
Epoch 1/10000
2986/2986 [==============================] - 4s 1ms/step - loss: 0.1865 - val_loss: 0.0732

Epoch 00001: val_loss did not improve
Epoch 2/10000
2986/2986 [==============================] - 2s 676us/step - loss: 0.0376 - val_loss: 0.0249

Epoch 00002: val_loss did not improve
Epoch 3/10000
2986/2986 [==============================] - 2s 685

In [ ]:
Model_Hybrid.load_weights("Model/weights.hdf5")

In [38]:
test_1_Pre =  Model_Hybrid.predict([test_label.drop(['P_ID'],axis=1),test_log])
Pre_df = pd.DataFrame(test_1_Pre)
Pre_df = pd.DataFrame(sc.inverse_transform(Pre_df))
W = [0.249,0.330,0.172,0.204]
Wmae = 0
for n in range(4):
    MAE = 0
    for i in range(len(Pre_df[0])):
        MAE = MAE + abs(Pre_df[n][i]-test_feature[TargetIndex[n]].iloc[i])
#     print(MAE/len(Pre_df[n]))
    Wmae = Wmae + (MAE/len(Pre_df[n]))*W[n]
print(Wmae)

0.0009370284199366616


In [50]:
sub_1_Pre =  Model_Hybrid.predict([sub_label.drop(['P_ID'],axis=1),sub_log])
Pre_df = pd.DataFrame(sub_1_Pre)
Pre_df = pd.DataFrame(sc.inverse_transform(Pre_df))


In [52]:
writer = pd.ExcelWriter('subdnnreport.xlsx')
Pre_df.to_excel(writer,sheet_name="1")
writer.save()